In [ ]:
!mkdir -p /tmp/pip/cache/
!cp ../input/pyroppl-152/pyro_ppl-1.5.2-py3-none-any.whl /tmp/pip/cache/
!cp ../input/pyroppl-152/pyro_api-0.1.2-py3-none-any.whl /tmp/pip/cache/
!pip install --no-index --find-links /tmp/pip/cache/ pyro-ppl==1.5.2

In [ ]:
!cp -R '/kaggle/input/yolov5/torch/root/.cache/torch' '/root/.cache/torch'
!cp -R '/kaggle/input/yolov5/ultralytics/root/.config/Ultralytics' '/root/.config/Ultralytics'

In [ ]:
import sys
sys.path.append("../input/timmmaster/")
sys.path.append("../input/ttach-kaggle/ttach/")
sys.path.append("../input/pythonbox/")

In [ ]:
import os
import warnings
from pprint import pprint
from glob import glob
from tqdm import tqdm
import gc

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torchvision.transforms as T
from box import Box
from timm import create_model
from sklearn.model_selection import StratifiedKFold
from torchvision.io import read_image
from torch.utils.data import DataLoader, Dataset
import math

import pytorch_lightning as pl
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning import callbacks
from pytorch_lightning.callbacks.progress import ProgressBarBase
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import LightningDataModule, LightningModule

from sklearn.linear_model import Ridge
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
import yaml
from sklearn.preprocessing import normalize

warnings.filterwarnings("ignore")

In [ ]:
# asthetics
import warnings
import sklearn.exceptions
warnings.filterwarnings('ignore')

#general
import pickle
from tqdm.auto import tqdm
from collections import defaultdict
import os
import numpy as np
import pandas as pd
import random
import gc
import cv2
import imageio
from itertools import product
gc.enable()
import glob
pd.set_option('display.max_columns', None) 
from PIL import Image

# visualization
import matplotlib.pyplot as plt
%matplotlib inline

# augmentation
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

# deep learning
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, OneCycleLR, CosineAnnealingLR
import torch
import torchvision
import timm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# metrics
from sklearn.metrics import mean_squared_error

# cv
from sklearn.model_selection import KFold

import glob, re

In [ ]:
from fastai.vision.all import *
import fastai

In [ ]:
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau
import albumentations
import imageio
import itertools

import random

import torchvision.transforms as T
from torchvision.io import read_image

import timm
import cv2

from torch.cuda.amp import autocast, GradScaler

# CatBoost
import catboost as cb
from catboost import CatBoost, Pool

import warnings 
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import cuml, pickle
from cuml.svm import SVR
print('RAPIDS version',cuml.__version__,'\n')

LOAD_SVR_FROM_PATH = None
LOAD_GPR_FROM_PATH = None

In [ ]:
smoke_test = ('CI' in os.environ)  # ignore; used to check code integrity in the Pyro repo
# assert pyro.__version__.startswith('1.6.0')
device = torch.device('cuda:0')

In [ ]:
import pyro
import pyro.contrib.gp as gp

# reference: https://pyro.ai/examples/gp.html
def train_GPR(embeddings, target, gpr_path, emb_path, y_path):
    """
    GPR training: The GPR model takes in the embeddings and the label/target of the samples
        embeddings: numpy array of shape(N, D)
        target: numpy array of shape (N, )
        
        gpr_path: the location to save the gpr model
        emb_path: the location to save the transformer oof embedding
        y_path: the location to save the original label/target
    """
    pyro.set_rng_seed(0)
    pyro.clear_param_store()
    no_steps = 300
    ls = 70.
    var = 420.
    noise = 1.

    X = torch.tensor(embeddings, dtype=torch.float32).to(device)
    y = torch.tensor(target, dtype=torch.float32).to(device)
    kernel = gp.kernels.RBF(input_dim=X.shape[1], variance=torch.tensor(var),
                        lengthscale=torch.tensor(ls)).to(device)

    gpr = gp.models.GPRegression(X, y, kernel, noise=torch.tensor(noise)).to(device)

    optimizer = torch.optim.Adam(gpr.parameters(), lr=0.0005)
    loss_fn = pyro.infer.Trace_ELBO().differentiable_loss
    losses = []
    num_steps = no_steps if not smoke_test else 2
    for i in tqdm(range(num_steps)):
        optimizer.zero_grad()
        loss = loss_fn(gpr.model, gpr.guide)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    
    with torch.no_grad():
        mean, cov = gpr(X, full_cov=True, noiseless=False)
    
    gpr_prediction = mean.cpu().numpy()
    
    print("Saving gpr model, embeddings, and target")
    save_gpr_model(gpr, embeddings, y, gpr_path, emb_path, y_path)
    
    del gpr, kernel
    torch.cuda.empty_cache()
    gc.collect()
    return gpr_prediction


def save_gpr_model(gpr, embedding, y, gpr_path, embedding_path, y_path):
    """
    To save the gpr model, embeddings and the original label required to perform inference for new test points.
        gpr: pyro gpr model
        embedding: transformer oof embedding
        y: original label/target
        
        gpr_path: the filepath to save the gpr model
        embedding_path: the filepath to save the transformer oof embedding
        y_path: the filepath to save the original label/target
        
    """    
    torch.save(gpr.state_dict(), gpr_path)
    np.save(embedding_path, embedding)
    np.save(y_path, y.cpu())
    print("Finished saving gpr weights at: {}".format(gpr_path))
    print("Finished saving embeddings at: {}".format(embedding_path))
    print("Finished saving embeddings label/target at: {}".format(y_path))

def get_pyro_emb_preds(embedding_path, y_new_path, gpr_model_path, test_embedding, is_normalize=False):
    pyro.clear_param_store()

    emb_train = np.load(embedding_path, allow_pickle=True)
    emb_train = np.vstack(emb_train)
    if is_normalize:
        X = torch.tensor(normalize(emb_train), dtype=torch.float32).cuda()
    else:
        X = torch.tensor((emb_train), dtype=torch.float32).cuda()
        
    y_new =  torch.tensor(np.load(y_new_path), dtype=torch.float32).cuda()
 
    emb_test = np.vstack(test_embedding)
    if is_normalize:
        X_test = torch.tensor(normalize(emb_test), dtype=torch.float32).cuda()
    else:
        X_test = torch.tensor((emb_test), dtype=torch.float32).cuda()
        
    kernel = gp.kernels.RBF(input_dim=X.shape[1], variance=torch.tensor(420.),
                    lengthscale=torch.tensor(70.)).cuda()
    gpr = gp.models.GPRegression(X, y_new, kernel, noise=torch.tensor(1.)).cuda()
    gpr.load_state_dict(torch.load(gpr_model_path, map_location='cuda:0'))
    
    with torch.no_grad():
        mean, cov = gpr(X_test, full_cov=True, noiseless=False)
    mean = np.array(mean.cpu())
    
    del gpr, kernel
    torch.cuda.empty_cache()
    gc.collect()
    return mean

In [ ]:
class Config:
    model_name = "swint_large224"
    data_dir = "../input/petfinder-pawpularity-score/"
    model_dirs = "../input/pawpularity-swin-new-exp1/"
    output_dir = "."
    img_train_dir = os.path.join(data_dir, "train")
    img_test_dir = os.path.join(data_dir, "test")
    random_seed = 555
    tta_times = 6 # 1: no TTA
    tta_beta = 1 / tta_times
    model_path = "swin_large_patch4_window7_224_in22k"
    pretrained = False
    n_fold = 5
    inp_channels = 3
    im_size =  224
    batch_size = 8
    num_workers = 0 # >0: OS Error
    out_features = 0
    dropout = 0
    scheduler_name = "OneCycleLR" #OneCycleLR

In [ ]:
class Config2:
    model_name = "swint_large224"
    data_dir = "../input/petfinder-pawpularity-score/"
    model_dirs = "../input/pawpularity-swin-new-exp4/"
    output_dir = "."
    img_train_dir = os.path.join(data_dir, "train")
    img_test_dir = os.path.join(data_dir, "test")
    random_seed = 555
    tta_times = 6 # 1: no TTA
    tta_beta = 1 / tta_times
    model_path = "swin_large_patch4_window7_224"
    pretrained = False
    n_fold = 5
    inp_channels = 3
    im_size =  224
    batch_size = 8
    num_workers = 0 # >0: OS Error
    out_features = 0
    dropout = 0
    scheduler_name = "OneCycleLR" #OneCycleLR

In [ ]:
class Config3:
    model_name = "swint_large224"
    data_dir = "../input/petfinder-pawpularity-score/"
    model_dirs = "../input/pawpularity-swin-new-exp3/"
    output_dir = "."
    img_train_dir = os.path.join(data_dir, "train")
    img_test_dir = os.path.join(data_dir, "test")
    random_seed = 555
    tta_times = 6 # 1: no TTA
    tta_beta = 1 / tta_times
    model_path = "swin_large_patch4_window7_224"
    pretrained = False
    n_fold = 5
    inp_channels = 3
    im_size = 224
    batch_size = 8
    num_workers = 0 # >0: OS Error
    out_features = 0
    dropout = 0
    scheduler_name = "OneCycleLR" #OneCycleLR

In [ ]:
def seed_everything(seed=Config.random_seed):
    os.environ['PYTHONSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic =True
    torch.backends.cudnn.benchmark =True

seed_everything()
# device optimization
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(f'Using device: {device}')

In [ ]:
IMAGENET_MEAN = [0.485, 0.456, 0.406]  # RGB
IMAGENET_STD = [0.229, 0.224, 0.225]  # RGB

def get_train_transforms(epoch, dim = Config.im_size):
    return A.Compose(
        [             
            # resize like Resize in fastai
            A.SmallestMaxSize(max_size=dim, p=1.0),
            A.CenterCrop(height=dim, width=dim, p=1.0),
            #A.HorizontalFlip(p = 0.5),
            #A.geometric.transforms.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15, p=1.0)
        ]
  )

def get_tta_transforms(epoch, dim = Config.im_size):
    return A.Compose(
        [             
            # resize like Resize in fastai
            A.SmallestMaxSize(max_size=dim, p=1.0),
            A.CenterCrop(height=dim, width=dim, p=1.0),
            A.HorizontalFlip(p = 0.5),
            A.geometric.transforms.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15, p=1.0)
        ]
  )

In [ ]:
class PetDataset(Dataset):
    def __init__(self, image_filepaths, targets, transform=None):
        self.image_filepaths = image_filepaths
        self.targets = targets
        self.transform = transform
    
    def __len__(self):
        return len(self.image_filepaths)

    def __getitem__(self, idx):
        image_filepath = self.image_filepaths[idx]
        with open(image_filepath, 'rb') as f:
            image = Image.open(f)
            image_rgb = image.convert('RGB')
        image = np.array(image_rgb)

        if self.transform is not None:
            image = self.transform(image = image)["image"]
        
        image = image / 255 # convert to 0-1
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        target = self.targets[idx]

        image = torch.tensor(image, dtype = torch.float)
        target = torch.tensor(target, dtype = torch.float)
        return image, target

In [ ]:
class MetricMonitor:
    def __init__(self, float_precision=3):
        self.float_precision = float_precision
        self.reset()

    def reset(self):
        self.metrics = defaultdict(lambda: {"val": 0, "count": 0, "avg": 0})

    def update(self, metric_name, val):
        metric = self.metrics[metric_name]

        metric["val"] += val
        metric["count"] += 1
        metric["avg"] = metric["val"] / metric["count"]

    def __str__(self):
        return " | ".join(
            [
                "{metric_name}: {avg:.{float_precision}f}".format(
                    metric_name=metric_name, avg=metric["avg"],
                    float_precision=self.float_precision
                )
                for (metric_name, metric) in self.metrics.items()
            ]
        )
    
def usr_rmse_score(output, target):
    y_pred = torch.sigmoid(output).cpu()
    y_pred = y_pred.detach().numpy()*100
    target = target.cpu()*100
    
    return mean_squared_error(target, y_pred, squared=False)

In [ ]:
class PetNet(nn.Module):
    def __init__(
        self,
        model_name = Config.model_path,
        out_features = Config.out_features,
        inp_channels=Config.inp_channels,
        pretrained=Config.pretrained
    ):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, in_chans=inp_channels, num_classes = out_features)
        self.fc = nn.Sequential(
            nn.Dropout(0.5), 
            nn.Linear(self.model.num_features, 1)
        )

    def extract_features(self, x):
        f = self.model(x)
        return f

    def forward(self, x):
        f = self.extract_features(x)
        out = self.fc(f)
        return out

In [ ]:
def petfinder_rmse(input,target):
    return 100*torch.sqrt(F.mse_loss(F.sigmoid(input.flatten()), target))

In [ ]:
class PetModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = timm.create_model('swin_large_patch4_window7_224', pretrained=False, in_chans=3, num_classes=0)
        self.fc = self.fc = nn.Sequential(
            nn.Dropout(0.5), 
            nn.Linear(self.backbone.num_features, 1)
        )

    def extract_features(self, x):
        f = self.backbone(x)
        return f

    def forward(self, x):
        f = self.extract_features(x)
        out = self.fc(f)
        return out

In [ ]:
img_train_dir = os.path.join(Config.data_dir, 'train')
def return_imgfilepath(name, folder=img_train_dir):
    path = os.path.join(folder, f'{name}.jpg')
    return path

train_file_path = os.path.join(Config.data_dir, 'train.csv')
train_df = pd.read_csv(train_file_path)

# set image filepath
train_df['file_path'] = train_df['Id'].apply(lambda x: return_imgfilepath(x))
train_df.head()

In [ ]:
ids = train_df['Id'].values
train_filepaths = train_df['file_path'].values
targets = train_df['Pawpularity'].values
num_bins = int(np.floor(1+(3.3)*(np.log2(len(train_df)))))
target_bins = pd.cut(targets/100, bins=num_bins, labels=False)

In [ ]:
test_file_path = os.path.join(Config.data_dir, 'test.csv')
test_df = pd.read_csv(test_file_path)
test_df['file_path'] = test_df['Id'].apply(lambda x: return_imgfilepath(x, folder=Config.img_test_dir))
target = 'Pawpularity'
test_filepaths = test_df['file_path'].values

In [ ]:
super_final_predictions = [] #classifier
super_final_predictions2 = [] #SVR
super_final_predictions3 = [] #GPR

skf = StratifiedKFold(n_splits = Config.n_fold, shuffle=True, random_state=Config.random_seed)
for i_fold, (train_idx, valid_idx) in enumerate(skf.split(train_filepaths, target_bins)):
    print(f'=== fold {i_fold}: training ===')
    """
    separate train/valid data 
    """
    X_train_paths = train_filepaths[train_idx]
    y_train = targets[train_idx]
    X_valid_paths = train_filepaths[valid_idx]
    y_valid = targets[valid_idx]
    valid_ids = ids[valid_idx]
    """
    prepare dataset
    """
    train_dataset = PetDataset(
      image_filepaths = X_train_paths,
      targets = y_train / 100,
      transform = get_train_transforms(0)
    )
    train_loader = DataLoader(
      train_dataset,
      batch_size = Config.batch_size,
      shuffle = False,
      num_workers = Config.num_workers,
      pin_memory = True
    )
    valid_dataset = PetDataset(
      image_filepaths = X_valid_paths,
      targets = y_valid / 100,
      transform = get_tta_transforms(0)
    )
    valid_loader = DataLoader(
      valid_dataset,
      batch_size = Config.batch_size,
      shuffle = False,
      num_workers = Config.num_workers,
      pin_memory = True
    )
    test_dataset = PetDataset(
      image_filepaths = test_filepaths,
      targets = np.zeros(len(test_filepaths)),
      transform = get_tta_transforms(0)
    )
    test_loader = DataLoader(
      test_dataset,
      batch_size = Config.batch_size,
      shuffle = False,
      num_workers = Config.num_workers,
      pin_memory = True
    )
    """
    instantiate model, cost function and optimizer
    """
    model = PetNet()
    model.load_state_dict(torch.load(Config.model_dirs+Config.model_path+f"_fold{i_fold}.pth"))
    model = model.to(device)
    model.eval()
    criterion = nn.BCEWithLogitsLoss()
    
    LOAD_SVR_FROM_PATH = "../input/paws-svr-gpr-new-2/"
    LOAD_GPR_FROM_PATH = "../input/paws-svr-gpr-new-2/"
    name = f"SVR_fold_{i_fold}.pkl"
    ##################
    # LOAD RAPIDS SVR 
    print('Loading SVR...',LOAD_SVR_FROM_PATH+name)
    clf = pickle.load(open(LOAD_SVR_FROM_PATH+name, "rb"))

    ##################
    # TEST PREDICTIONS
    print('Predicting test...')
    final_test_predictions_all = []
    final_test_predictions_all2 = []
    final_test_predictions_all3 = []
    for _ in range(5):
        final_test_predictions = []
        embed_test = np.array([]).reshape((0,1536))
        stream = tqdm(test_loader)
        for i, (images, target) in enumerate(stream, start = 1):
            images = images.to(device, non_blocking = True).float()
            target = target.to(device, non_blocking = True).float().view(-1, 1)
            with torch.no_grad():
                output = model(images)
                emb = model.extract_features(images)
            embed_test = np.concatenate([embed_test, emb.tolist()])
            pred = (torch.sigmoid(output).detach().cpu().numpy() * 100).ravel().tolist()
            final_test_predictions.extend(pred)
        final_test_predictions2 = clf.predict(normalize(embed_test))
        final_test_predictions3 = get_pyro_emb_preds(LOAD_GPR_FROM_PATH+f'notebook_embeddings_f{i_fold}.npy', LOAD_GPR_FROM_PATH+f'notebook_y_f{i_fold}.npy', 
                                                     LOAD_GPR_FROM_PATH+f'notebook_gpr_f{i_fold}', normalize(embed_test))
        final_test_predictions_all.append(final_test_predictions)
        final_test_predictions_all2.append(final_test_predictions2)
        final_test_predictions_all3.append(final_test_predictions3)
    final_test_predictions_all = np.mean(final_test_predictions_all, axis=0)
    final_test_predictions_all2 = np.mean(final_test_predictions_all2, axis=0)
    final_test_predictions_all3 = np.mean(final_test_predictions_all3, axis=0)
    super_final_predictions.append(final_test_predictions_all)
    super_final_predictions2.append(final_test_predictions_all2)
    super_final_predictions3.append(final_test_predictions_all3)
    ##################
    
    del model, clf
    del train_loader, train_dataset
    del test_loader, test_dataset
    del valid_loader, valid_dataset
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
super_final_predictions_2 = [] #classifier
super_final_predictions2_2 = [] #SVR
super_final_predictions3_2 = [] #GPR

skf = StratifiedKFold(n_splits = Config.n_fold, shuffle=True, random_state=Config.random_seed)
for i_fold, (train_idx, valid_idx) in enumerate(skf.split(train_filepaths, target_bins)):
    print(f'=== fold {i_fold}: training ===')
    """
    separate train/valid data 
    """
    X_train_paths = train_filepaths[train_idx]
    y_train = targets[train_idx]
    X_valid_paths = train_filepaths[valid_idx]
    y_valid = targets[valid_idx]
    valid_ids = ids[valid_idx]
    """
    prepare dataset
    """
    train_dataset = PetDataset(
      image_filepaths = X_train_paths,
      targets = y_train / 100,
      transform = get_train_transforms(0)
    )
    train_loader = DataLoader(
      train_dataset,
      batch_size = Config.batch_size,
      shuffle = False,
      num_workers = Config.num_workers,
      pin_memory = True
    )
    valid_dataset = PetDataset(
      image_filepaths = X_valid_paths,
      targets = y_valid / 100,
      transform = get_tta_transforms(0)
    )
    valid_loader = DataLoader(
      valid_dataset,
      batch_size = Config.batch_size,
      shuffle = False,
      num_workers = Config.num_workers,
      pin_memory = True
    )
    test_dataset = PetDataset(
      image_filepaths = test_filepaths,
      targets = np.zeros(len(test_filepaths)),
      transform = get_tta_transforms(0)
    )
    test_loader = DataLoader(
      test_dataset,
      batch_size = Config.batch_size,
      shuffle = False,
      num_workers = Config.num_workers,
      pin_memory = True
    )
    """
    instantiate model, cost function and optimizer
    """
    model = PetNet()
    model.load_state_dict(torch.load(Config2.model_dirs+Config2.model_path+f"_fold{i_fold}.pth"))
    model = model.to(device)
    model.eval()
    criterion = nn.BCEWithLogitsLoss()
    
    LOAD_SVR_FROM_PATH = "../input/paws-svr-gpr-new-5/"
    LOAD_GPR_FROM_PATH = "../input/paws-svr-gpr-new-5/"
    name = f"SVR_fold_{i_fold}.pkl"
    ##################
    # LOAD RAPIDS SVR 
    print('Loading SVR...',LOAD_SVR_FROM_PATH+name)
    clf = pickle.load(open(LOAD_SVR_FROM_PATH+name, "rb"))

    ##################
    # TEST PREDICTIONS
    print('Predicting test...')
    final_test_predictions_all = []
    final_test_predictions_all2 = []
    final_test_predictions_all3 = []
    for _ in range(5):
        final_test_predictions = []
        embed_test = np.array([]).reshape((0,1536))
        stream = tqdm(test_loader)
        for i, (images, target) in enumerate(stream, start = 1):
            images = images.to(device, non_blocking = True).float()
            target = target.to(device, non_blocking = True).float().view(-1, 1)
            with torch.no_grad():
                output = model(images)
                emb = model.extract_features(images)
            embed_test = np.concatenate([embed_test, emb.tolist()])
            pred = (torch.sigmoid(output).detach().cpu().numpy() * 100).ravel().tolist()
            final_test_predictions.extend(pred)
        final_test_predictions2 = clf.predict(normalize(embed_test))
        final_test_predictions3 = get_pyro_emb_preds(LOAD_GPR_FROM_PATH+f'notebook_embeddings_f{i_fold}.npy', LOAD_GPR_FROM_PATH+f'notebook_y_f{i_fold}.npy', 
                                                     LOAD_GPR_FROM_PATH+f'notebook_gpr_f{i_fold}', normalize(embed_test))
        final_test_predictions_all.append(final_test_predictions)
        final_test_predictions_all2.append(final_test_predictions2)
        final_test_predictions_all3.append(final_test_predictions3)
    final_test_predictions_all = np.mean(final_test_predictions_all, axis=0)
    final_test_predictions_all2 = np.mean(final_test_predictions_all2, axis=0)
    final_test_predictions_all3 = np.mean(final_test_predictions_all3, axis=0)
    super_final_predictions_2.append(final_test_predictions_all)
    super_final_predictions2_2.append(final_test_predictions_all2)
    super_final_predictions3_2.append(final_test_predictions_all3)
    ##################
    
    del model, clf
    del train_loader, train_dataset
    del test_loader, test_dataset
    del valid_loader, valid_dataset
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
super_final_predictions_3 = [] #classifier
super_final_predictions2_3 = [] #SVR
super_final_predictions3_3 = [] #GPR

skf = StratifiedKFold(n_splits = Config.n_fold, shuffle=True, random_state=Config.random_seed)
for i_fold, (train_idx, valid_idx) in enumerate(skf.split(train_filepaths, target_bins)):
    print(f'=== fold {i_fold}: training ===')
    """
    separate train/valid data 
    """
    X_train_paths = train_filepaths[train_idx]
    y_train = targets[train_idx]
    X_valid_paths = train_filepaths[valid_idx]
    y_valid = targets[valid_idx]
    valid_ids = ids[valid_idx]
    """
    prepare dataset
    """
    train_dataset = PetDataset(
      image_filepaths = X_train_paths,
      targets = y_train / 100,
      transform = get_train_transforms(0)
    )
    train_loader = DataLoader(
      train_dataset,
      batch_size = Config.batch_size,
      shuffle = False,
      num_workers = Config.num_workers,
      pin_memory = True
    )
    valid_dataset = PetDataset(
      image_filepaths = X_valid_paths,
      targets = y_valid / 100,
      transform = get_tta_transforms(0)
    )
    valid_loader = DataLoader(
      valid_dataset,
      batch_size = Config.batch_size,
      shuffle = False,
      num_workers = Config.num_workers,
      pin_memory = True
    )
    test_dataset = PetDataset(
      image_filepaths = test_filepaths,
      targets = np.zeros(len(test_filepaths)),
      transform = get_tta_transforms(0)
    )
    test_loader = DataLoader(
      test_dataset,
      batch_size = Config.batch_size,
      shuffle = False,
      num_workers = Config.num_workers,
      pin_memory = True
    )
    """
    instantiate model, cost function and optimizer
    """
    model = PetNet()
    model.load_state_dict(torch.load(Config3.model_dirs+Config3.model_path+f"_fold{i_fold}.pth"))
    model = model.to(device)
    model.eval()
    criterion = nn.BCEWithLogitsLoss()
    
    LOAD_SVR_FROM_PATH = "../input/paws-svr-gpr-new-4/"
    LOAD_GPR_FROM_PATH = "../input/paws-svr-gpr-new-4/"
    name = f"SVR_fold_{i_fold}.pkl"
    ##################
    # LOAD RAPIDS SVR 
    print('Loading SVR...',LOAD_SVR_FROM_PATH+name)
    clf = pickle.load(open(LOAD_SVR_FROM_PATH+name, "rb"))

    ##################
    # TEST PREDICTIONS
    print('Predicting test...')
    final_test_predictions_all = []
    final_test_predictions_all2 = []
    final_test_predictions_all3 = []
    for _ in range(5):
        final_test_predictions = []
        embed_test = np.array([]).reshape((0,1536))
        stream = tqdm(test_loader)
        for i, (images, target) in enumerate(stream, start = 1):
            images = images.to(device, non_blocking = True).float()
            target = target.to(device, non_blocking = True).float().view(-1, 1)
            with torch.no_grad():
                output = model(images)
                emb = model.extract_features(images)
            embed_test = np.concatenate([embed_test, emb.tolist()])
            pred = (torch.sigmoid(output).detach().cpu().numpy() * 100).ravel().tolist()
            final_test_predictions.extend(pred)
        final_test_predictions2 = clf.predict(normalize(embed_test))
        final_test_predictions3 = get_pyro_emb_preds(LOAD_GPR_FROM_PATH+f'notebook_embeddings_f{i_fold}.npy', LOAD_GPR_FROM_PATH+f'notebook_y_f{i_fold}.npy', 
                                                     LOAD_GPR_FROM_PATH+f'notebook_gpr_f{i_fold}', normalize(embed_test))
        final_test_predictions_all.append(final_test_predictions)
        final_test_predictions_all2.append(final_test_predictions2)
        final_test_predictions_all3.append(final_test_predictions3)
    final_test_predictions_all = np.mean(final_test_predictions_all, axis=0)
    final_test_predictions_all2 = np.mean(final_test_predictions_all2, axis=0)
    final_test_predictions_all3 = np.mean(final_test_predictions_all3, axis=0)
    super_final_predictions_3.append(final_test_predictions_all)
    super_final_predictions2_3.append(final_test_predictions_all2)
    super_final_predictions3_3.append(final_test_predictions_all3)
    ##################
    
    del model, clf
    del train_loader, train_dataset
    del test_loader, test_dataset
    del valid_loader, valid_dataset
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
super_final_preds = []
df = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
df['Path'] = df['Id'].map(lambda x:'../input/petfinder-pawpularity-score/train/'+str(x)+'.jpg')

df_test = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
df_test['Path'] = df_test['Id'].map(lambda x:'../input/petfinder-pawpularity-score/test/'+str(x)+'.jpg')

df_test['norm_score'] = 0.01

skf = StratifiedKFold(
    n_splits=5, shuffle=True, random_state=555
)
num_bins = int(np.floor(1+(3.3)*(np.log2(len(df)))))
df['norm_score'] = df['Pawpularity']/100
df['bins'] = pd.cut(df['norm_score'].values, bins=num_bins, labels=False)

for fold_, (train_idx, val_idx) in enumerate(skf.split(df["Id"], df["bins"])):
    print('#'*25)
    print('### FOLD',fold_+1)
    print('#'*25)
        
    df_train = df.loc[train_idx].reset_index(drop=True)
    df_val = df.loc[val_idx].reset_index(drop=True)
    
    df['is_valid'] = df.index.map(lambda x: x in val_idx)
    dls = ImageDataLoaders.from_df(df,
                               valid_col='is_valid',
                               seed=555,
                               fn_col='Path',
                               label_col='norm_score',
                               y_block=RegressionBlock,
                               bs=8,
                               shuffle=False,
                               num_workers=8,
                               item_tfms=Resize(224),
                               batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation()]))
    model = PetModel()
    learn = Learner(dls, model, loss_func=BCEWithLogitsLossFlat(), metrics=[AccumMetric(func=petfinder_rmse)])
    learn.model.load_state_dict(torch.load(f"../input/pawpularity-swin-l-fork/fold_{fold_}.pth")['model']) # Loading trained model weights
    learn.model.eval()
    learn.model.cuda()

    ##################
    # TEST PREDICTIONS
    print('Predicting test...')
    
    test_dl = dls.test_dl(df_test)
    preds, _ = learn.tta(dl=test_dl, n=5, beta=0)
    super_final_preds.append(preds)
    ##################
    
    ##################
    # COMPUTE RSME
    del model
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
params_l = {'max_depth': 2,
 'objective': 'regression',
 'metric': 'rmse',
 'learning_rate': 0.05,
 'seed': 999,
 'feature_pre_filter': False,
 'lambda_l1': 0.29742200421119536,
 'lambda_l2': 5.720025931478957,
 'num_leaves': 3,
 'feature_fraction': 0.748,
 'bagging_fraction': 0.8840460329244646,
 'bagging_freq': 4,
 'min_child_samples': 100}

params_r = {"alpha": 10, "random_state": 999}

params_cat = {
        'loss_function' : 'RMSE',
        'eval_metric' : 'RMSE',
        'iterations' : 1000,
        'grow_policy' : 'SymmetricTree',
        'random_state' : 42,
        'depth': 4,
        'learning_rate': 0.01943558754716122,
        'random_strength': 97,
        'bagging_temperature': 2.9049665973532472,
        'od_type': 'Iter',
        'od_wait': 45}

In [ ]:
def add_labels(df):
    df[['cat', 'dog', 'neither']] = 0
    ohe = []
    for idx, path in zip(itertools.count(), df['path'].values):
        img = imageio.imread(path)
        result = yolov5(img)
        labels = result.pandas().xyxy[0]['name'].values
        found_label = False
        for label in labels:
            if label == 'cat':
                ohe.append([1,0,0])
                found_label = True
                break
            elif label == 'dog':
                ohe.append([0,1,0])
                found_label = True
                break
        if not found_label:
            ohe.append([0,0,1])
            
    df[['cat', 'dog', 'neither']] = ohe
    return df

yolov5 = torch.hub.load('ultralytics/yolov5', 'yolov5x6')

In [ ]:
X_train = pd.read_csv("../input/pawpularity-for-post-process-v2/paws_for_post_process_v3.csv")
y_train = X_train.Pawpularity.values
kfold_array = X_train.kfold.values
X_train.drop(["Id", "kfold", "Pawpularity"], axis=1, inplace=True)

X_test = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
super_final_predictions = np.mean(np.column_stack(super_final_predictions), axis=1)
super_final_predictions2 = np.mean(np.column_stack(super_final_predictions2), axis=1)
super_final_predictions3 = np.mean(np.column_stack(super_final_predictions3), axis=1)
super_final_predictions_2 = np.mean(np.column_stack(super_final_predictions_2), axis=1)
super_final_predictions2_2 = np.mean(np.column_stack(super_final_predictions2_2), axis=1)
super_final_predictions3_2 = np.mean(np.column_stack(super_final_predictions3_2), axis=1)
super_final_predictions_3 = np.mean(np.column_stack(super_final_predictions_3), axis=1)
super_final_predictions2_3 = np.mean(np.column_stack(super_final_predictions2_3), axis=1)
super_final_predictions3_3 = np.mean(np.column_stack(super_final_predictions3_3), axis=1)
super_final_preds = np.mean(np.hstack(super_final_preds), axis=1) * 100
X_test["preds1"] = super_final_predictions
X_test["preds2"] = super_final_predictions2
X_test["preds3"] = super_final_predictions3
X_test["preds1_2"] = super_final_predictions_2
X_test["preds2_2"] = super_final_predictions2_2
X_test["preds3_2"] = super_final_predictions3_2
X_test["preds1_3"] = super_final_predictions_3
X_test["preds2_3"] = super_final_predictions2_3
X_test["preds3_3"] = super_final_predictions3_3
X_test["preds"] = super_final_preds
X_test['path'] = X_test['Id'].map(lambda x:f'../input/petfinder-pawpularity-score/test/{str(x)}'+'.jpg')
X_test = add_labels(X_test)
X_test = X_test.drop(["Id", "path"], axis=1)

del yolov5
torch.cuda.empty_cache()
gc.collect()

In [ ]:
y_preds_l = []
models_l = []
oof_train_l = np.zeros((len(X_train)))
y_preds_cat = []
models_cat = []
oof_train_cat = np.zeros((len(X_train)))
y_preds_r = []
models_r = []
oof_train_r = np.zeros((len(X_train)))
for fold_ in range(10):
    train_index = np.nonzero(kfold_array != fold_)[0]
    valid_index = np.nonzero(kfold_array == fold_)[0]
    X_tr = X_train.iloc[train_index, :]
    y_tr = y_train[train_index]
    X_val = X_train.iloc[valid_index, :]
    y_val = y_train[valid_index]
    categorical_cols = ["Subject Focus", "Eyes", "Face", "Near", "Action", "Accessory", "Group", "Collage", "Human", "Occlusion", "Info", "Blur", 
                        "cat", "dog", "neither"]
    lgb_train = lgb.Dataset(X_tr, y_tr, categorical_feature=categorical_cols)
    lgb_eval = lgb.Dataset(
        X_val, y_val, reference=lgb_train, categorical_feature=categorical_cols
    )
    model_l = lgb.train(
        params_l,
        lgb_train,
        valid_sets=[lgb_train, lgb_eval],
        verbose_eval=30,
        num_boost_round=1000,
        early_stopping_rounds=50,
    )
    oof_train_l[valid_index] = model_l.predict(X_val, num_iteration=model_l.best_iteration)
    y_pred_l = model_l.predict(X_test, num_iteration=model_l.best_iteration)
    y_preds_l.append(y_pred_l)
    models_l.append(model_l)
    model_cat = cb.CatBoostRegressor(**params_cat)
    model_cat.fit(
        X_tr,
        y_tr,
        eval_set=[(X_val, y_val)],
        early_stopping_rounds=100,
        cat_features=categorical_cols,
        verbose=False,
    )
    oof_train_cat[valid_index] = model_cat.predict(X_val)
    y_pred_cat = model_cat.predict(X_test)
    y_preds_cat.append(y_pred_cat)
    models_cat.append(model_cat)
    model_r = Ridge(**params_r)
    model_r.fit(X_tr, y_tr)
    oof_train_r[valid_index] = model_r.predict(X_val)
    y_pred_r = model_r.predict(X_test)
    y_preds_r.append(y_pred_r)
    models_r.append(model_r)

In [ ]:
print(mean_squared_error(oof_train_l, y_train, squared=False))
y_sub_l = sum(y_preds_l) / len(y_preds_l)

print(mean_squared_error(oof_train_cat, y_train, squared=False))
y_sub_cat = sum(y_preds_cat) / len(y_preds_cat)

print(mean_squared_error(oof_train_r, y_train, squared=False))
y_sub_r = sum(y_preds_r) / len(y_preds_r)

In [ ]:
df_test = pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")
df_test["Pawpularity"] = y_sub_cat * 0.67 + y_sub_r * 0.33
df_test = df_test[["Id", "Pawpularity"]]
df_test.to_csv("submission.csv", index=False)
df_test.head()